# Análisis Tabla Accidentes enero - abril de 2023 

In [1]:
import os
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Extracción de datos de la BD SQL Server

In [2]:
class SQLServer:
    def __init__(self, server_name, database_name, user_name, password):
        self.server_name = server_name
        self.database_name = database_name
        self.user_name = user_name
        self.password = password
        self.connection_string = f'DRIVER={{SQL Server}};SERVER={self.server_name};DATABASE={self.database_name};UID={self.user_name};PWD={self.password}'
        self.connection = pyodbc.connect(self.connection_string)

    def query_to_dataframe(self, query):
            df = pd.read_sql(query, self.connection)
            return df

In [3]:
# Crea una instancia de la clase SQLServer
sql_server = SQLServer('TVM-PAPP02-VIC', 'AnalystBi', 'ajusta', 'Holading2022*')

In [4]:
# Define la consulta
query = "SELECT * FROM Ajuasta_ACCIDENTES_TBL WHERE Fecha_reporte BETWEEN '20230101' AND '20230430'"

In [5]:
# Obtiene un dataframe de la tabla 'nombre_tabla'
df = sql_server.query_to_dataframe(query)
df.head()

C:\Users\JUAN ACOSTA\AppData\Local\Temp\ipykernel_8176\517284428.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection)


,Numero,Documento,Placa,Fecha_reporte,Fecha_accidente,id_compania,nm_compania,cliente_cliente,nm_cliente,nm_creado,...,obs_hipotesis_agente,curos_hipotesis,cobertura_rcc,cobertura_rce,cobertura_rcce,cobertura_rcee,cobertura_tr,cobertura_plo,cobertura_ip,cobertura_tercero
0,166741,None,SNH155,2023-01-01,20/12/2022 1:30:00 a.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,891100299,COOP. DE TRANSPORTADORES DEL HUILA - COOTRANSH...,CAROLINA CHAVEZ,...,None,,None,"100 SMMLV,100 SMMLV,200 SMMLV",None,None,None,None,None,None
1,166742,91229335,SUD629,2023-01-01,20/11/2022 10:00:00 p.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,890211768,TAXIS DEL SUR S.A. - URBANO,CAROLINA CHAVEZ,...,CODUCTOR DE TAXI SE RETIRO DE LA CLINICA VOLUN...,Consecuencias penales y civiles en accidente d...,None,"100 SMMLV,100 SMMLV,200 SMMLV",None,None,None,None,None,None
2,166743,None,GUX253,2023-01-02,30/11/2022 1:44:00 p.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,900364615,ORGANIZACION SUMA S.A.S,LEIDY GARCIA ZAMORA,...,,,None,"60 SMMLV,60 SMMLV,120 SMMLV",None,None,None,None,None,None
3,166744,1024544074,WCM367,2023-01-02,13/10/2022 1:29:00 p.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,800145640,RADIO TAXI AUTOLAGOS,BRAIAN ALEJANDRO MORALES,...,None,,None,"80 SMMLV,80 SMMLV,160 SMMLV",None,None,None,None,None,None
4,166745,None,WHN603,2023-01-02,10/05/2022 12:00:00 p.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,891401326,TRANS SERVILUJO S.A.,TITO ALEJANDRO ORDÓÑEZ CORTEZ,...,None,Marco legal para conductores - Seguridad basad...,None,"60 SMMLV,60 SMMLV,120 SMMLV",None,None,None,None,None,None


## Limpieza de datos

In [6]:
df = df.iloc[:, [0] + list(range(2,13)) + list(range(15,27)) + [30, 32, 34, 35, 36, 38]]

In [7]:
df.head()

,Numero,Placa,Fecha_reporte,Fecha_accidente,id_compania,nm_compania,cliente_cliente,nm_cliente,nm_creado,tipo_accidente,...,valor_reserva,valor_reserva_actual,todo_riesgo,tipo_servicio,revision_especial,Hipotesis_agente,modelo,dia_accidente,hora_acidente,obs_hipotesis_agente
0,166741,SNH155,2023-01-01,20/12/2022 1:30:00 a.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,891100299,COOP. DE TRANSPORTADORES DEL HUILA - COOTRANSH...,CAROLINA CHAVEZ,CHOQUE SIMPLE,...,1201760,1201760.0,NO,INTERMUNICIPAL,NO,SIN RESPONSABILIDAD,1980,MARTES,00:00 A.M. - 05:59 A.M.,None
1,166742,SUD629,2023-01-01,20/11/2022 10:00:00 p.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,890211768,TAXIS DEL SUR S.A. - URBANO,CAROLINA CHAVEZ,CHOQUE SIMPLE Y LESIONES,...,9976000,7614240.0,NO,URBANO,NO,EMBRIAGUEZ APARENTE.,2007,DOMINGO,06:00 P.M. - 11:59 P.M.,CODUCTOR DE TAXI SE RETIRO DE LA CLINICA VOLUN...
2,166743,GUX253,2023-01-02,30/11/2022 1:44:00 p.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,900364615,ORGANIZACION SUMA S.A.S,LEIDY GARCIA ZAMORA,CHOQUE SIMPLE,...,2361760,2361760.0,NO,URBANO,NO,SIN HIPÓTESIS,2021,MIERCOLES,12:00 P.M. - 05:59 P.M.,
3,166744,WCM367,2023-01-02,13/10/2022 1:29:00 p.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,800145640,RADIO TAXI AUTOLAGOS,BRAIAN ALEJANDRO MORALES,CHOQUE SIMPLE Y HOMICIDIO,...,8120000,8120000.0,NO,URBANO,NO,SIN RESPONSABILIDAD,2014,JUEVES,12:00 P.M. - 05:59 P.M.,None
4,166745,WHN603,2023-01-02,10/05/2022 12:00:00 p.m.,860037013,COMPAÑIA MUNDIAL DE SEGUROS S.A.,891401326,TRANS SERVILUJO S.A.,TITO ALEJANDRO ORDÓÑEZ CORTEZ,CHOQUE SIMPLE,...,4149900,1508000.0,NO,URBANO,NO,ARRANCAR SIN PRECAUCIÓN.,2010,MARTES,12:00 P.M. - 05:59 P.M.,None


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4212 entries, 0 to 4211
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Numero                4212 non-null   object 
 1   Placa                 4212 non-null   object 
 2   Fecha_reporte         4212 non-null   object 
 3   Fecha_accidente       4212 non-null   object 
 4   id_compania           4212 non-null   object 
 5   nm_compania           4212 non-null   object 
 6   cliente_cliente       4212 non-null   object 
 7   nm_cliente            4212 non-null   object 
 8   nm_creado             4212 non-null   object 
 9   tipo_accidente        4212 non-null   object 
 10  nm_ciudad             4212 non-null   object 
 11  nm_departamento       4212 non-null   object 
 12  poliza_rcc            397 non-null    object 
 13  poliza_rce            3845 non-null   object 
 14  poliza_rcce           0 non-null      object 
 15  poliza_rcee          

In [9]:
# Convertir fechas de texto a formato date
df['Fecha_reporte'] = pd.to_datetime(df['Fecha_reporte'], format= '%Y-%m-%d')
df['Fecha_accidente'] = pd.to_datetime(df['Fecha_accidente'], infer_datetime_format=True)

In [11]:
# Convertir texto en datos float 
df['valor_reserva'] = pd.to_numeric(df['valor_reserva'], errors='coerce')
df['modelo'] = pd.to_numeric(df['modelo'], errors='coerce')


In [12]:
df.describe().style.format(precision=2)

,valor_reserva,valor_reserva_actual,modelo
count,4199.00,4199.00,4193.00
mean,2954726.95,2140662.76,2007.87
std,5808734.03,5392312.34,185.69
min,0.00,0.00,1.00
25%,556800.00,0.00,2010.00
50%,1740000.00,375840.00,2014.00
75%,3749120.00,2308400.00,2017.00
max,124360646.00,181394652.00,11111.00


### Fechas limpieza

### Modelos limpeiza

In [16]:
# Validación de modelos de vehículos
modelo_remplazo = {
    1:0,
    11:0,
    1111:0,
    11111:0,
    None:0,
    2222:0,
    2117:0,
    2111:0,
    2105:0,
    2041:0,
    1223:0,
    201:0,
    58:0,
    22:0,
    20:0
}

df['modelo'] = df['modelo'].replace(modelo_remplazo)
modelos_tf = df['modelo'].value_counts()

In [17]:
# Modelos sin datos o datos incoherentes:
modelos_tf[0]

36

In [20]:
# Eliminación de datos incoherentes en la columna Modelos
modelos_tf = modelos_tf.drop(0)